In [5]:
%load_ext autoreload
%autoreload 2
import sys
import torch

sys.path.insert(0,'/home/aw1223/ip/agile')

from sdk.ample import Ample

from torch_geometric.datasets import FakeDataset #TODO remove
from sdk.models.models import MLP_Model,Interaction_Net_Model





class ToyModel(torch.nn.Module):
    def __init__(self, in_channels=32, out_channels=32, layer_count=1, hidden_dimension=32, precision = torch.float32):
        super().__init__()
        self.precision = precision
        self.layers = torch.nn.ModuleList()


        self.linear1 = MLP_Model(in_channels, out_channels) 
        self.linear1.name  = 'linear_embedder1'
        self.layers.append(self.linear1) #Used to map weights in SDK


        self.linear2 = MLP_Model(in_channels, out_channels) 
        self.linear2.name  = 'linear_embedder2'
        self.layers.append(self.linear2) #Used to map weights in SDK

        # self.int_net = Interaction_Net_Model(in_channels, out_channels, layer_count, hidden_dimension, precision)
        # self.int_net.name  = 'int_net'
        # self.layers.append(self.int_net) #Used to map weights in SDK


        for layer in self.layers:
            layer.to(self.precision)

    def forward(self, x):
        outputs_model = []
        x = x.to(self.precision) 
        print(x) 
        outputs_sub_model1,x = self.linear1(x)
        print('outputs_sub_model lin 1')
        print(outputs_sub_model1)
        #need to add a name to each model output as order of computation is not guaranteed
        # outputs_model = outputs_model + outputs_sub_model #Add instead of append to have layer outputs in single list as tb iterates over each layer
        # print(x)

        outputs_sub_model2,x = self.linear2(x)
        print('outputs_sub_model lin 2')
        print(outputs_sub_model2)
        outputs_model = outputs_sub_model1 + outputs_sub_model2
        # _,x = self.int_net(x, edge_index, edge_attr)

        return outputs_model,x


model = ToyModel(32,32)

dataset = FakeDataset(
                        num_graphs=1, 
                        avg_num_nodes = 3,
                        avg_degree=3,
                        num_channels=32,
                        edge_dim=0
                    )[0]
# dataset.x = torch.tensor(dataset.x)
# External inputs {'int_net_input_1': None, 'int_net_input_2': None, 'linear_input_0': None}
#Need inputs to be sorted
print('num_nodes')
num_nodes  = len(dataset.x)
print(num_nodes)
print('num_edges')
print(dataset.num_edges)
num_edges = dataset.num_edges
inputs = [dataset.x]
outputs,x = model(*inputs)
print('outputs',outputs)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
num_nodes
3
num_edges
2
tensor([[ 9.6883, 11.1025,  9.8591,  8.0255,  9.6129,  9.1705,  8.4635,  8.1544,
          7.1679,  9.9791,  9.5058,  9.4450,  7.7037,  9.6371,  8.6707,  8.8444,
          9.9860,  7.3060,  9.7810,  8.6714,  8.9157,  8.2921,  8.6613,  9.8056,
         10.2106,  9.4653,  9.7499,  6.8711,  8.6038, 11.3398,  7.6130,  8.7171],
        [ 1.1612,  1.9904,  0.9164,  1.6336,  1.7422,  0.0230,  1.6457,  1.3090,
          0.3629,  3.8792, -0.5514,  0.3952, -0.5563,  0.7184,  0.9410,  1.0139,
          0.5513,  0.1030,  1.6343,  0.8491, -0.6704, -1.0388, -0.1935,  1.4806,
          0.9080,  0.4949,  0.9002,  1.6629,  1.8268,  0.4477,  1.7155,  1.9857],
        [ 5.5679,  5.9731,  7.2558,  4.9565,  4.6206,  6.4870,  5.1744,  6.0215,
          6.4554,  4.9583,  4.5069,  7.0936,  7.2768,  5.0003,  6.0884,  5.8601,
          4.9280,  5.6136,  4.8522,  5.6794,  4.5227,  6.5616,  6.6562,  7.3

In [6]:
# # print('dataset.x' ,dataset.x)
# # print('dataset.edge_index' ,dataset.edge_index)
# # print('dataset.edge_attr' ,dataset.edge_attr)


# dataset.x = tensor([[-2.0775e-01,  5.4138e-01, -9.3228e-01, -1.5647e-02, -9.2422e-01,
#          -1.4551e+00,  1.1447e+00,  4.8944e-01, -1.2190e+00, -2.1434e+00,
#           8.0335e-01, -1.3588e+00, -1.2911e+00,  1.1237e+00,  1.1250e-01,
#           3.9626e-01,  5.1411e-01,  9.9543e-01,  7.6991e-02,  1.1795e+00,
#          -1.2423e+00, -2.6467e-01, -4.9839e-01, -1.0298e+00, -2.2073e+00,
#           1.7572e+00, -5.1693e-01,  1.4884e+00,  1.1717e+00, -1.6047e+00,
#          -1.0268e+00,  1.6982e+00],
#         [-1.8529e+00,  1.6048e+00, -6.7576e-01,  4.8206e-01, -7.3550e-01,
#           2.7671e+00,  6.5742e-01,  1.1275e-01, -8.2426e-01,  9.4348e-01,
#          -5.4252e-01,  1.3881e+00, -3.2231e-01,  2.2466e+00,  2.9660e-01,
#          -3.3892e-01, -3.5646e-01,  1.7096e+00,  6.5559e-01,  9.3671e-01,
#          -2.0564e-01, -2.5100e-01, -6.0347e-02, -9.6708e-01,  2.2658e+00,
#           2.2228e-01, -2.8040e+00, -8.2614e-01,  5.2462e-01, -2.1695e+00,
#          -1.4330e-02,  8.4461e-01],
#         [ 6.7803e+00,  5.0186e+00,  5.3118e+00,  6.6033e+00,  5.3946e+00,
#           4.0241e+00,  5.3939e+00,  5.0030e+00,  4.8918e+00,  3.9511e+00,
#           6.0363e+00,  4.9824e+00,  5.2248e+00,  6.5227e+00,  5.0742e+00,
#           4.1273e+00,  4.4227e+00,  5.2284e+00,  5.5740e+00,  6.3545e+00,
#           4.3107e+00,  5.2994e+00,  5.3288e+00,  5.3148e+00,  5.8875e+00,
#           5.0108e+00,  5.8182e+00,  4.4662e+00,  5.2387e+00,  6.0119e+00,
#           4.5336e+00,  5.5987e+00],
#         [ 7.2144e+00,  6.8995e+00,  7.3504e+00,  6.8768e+00,  8.2005e+00,
#           5.3968e+00,  7.1683e+00,  8.5331e+00,  5.7610e+00,  7.1746e+00,
#           8.3445e+00,  7.0579e+00,  5.7996e+00,  6.3835e+00,  7.3726e+00,
#           7.4944e+00,  6.1401e+00,  5.5552e+00,  7.0648e+00,  6.5871e+00,
#           6.8504e+00,  7.1488e+00,  7.7956e+00,  5.9428e+00,  7.5479e+00,
#           9.2140e+00,  7.0638e+00,  6.7924e+00,  5.0496e+00,  8.2112e+00,
#           7.2045e+00,  7.2927e+00],
#         [ 8.7854e+00,  6.3623e+00,  5.6675e+00,  5.8517e+00,  6.0286e+00,
#           6.0502e+00,  7.9339e+00,  7.0228e+00,  8.4109e+00,  8.4846e+00,
#           6.2293e+00,  6.7835e+00,  6.1736e+00,  6.1607e+00,  5.8996e+00,
#           6.4516e+00,  5.5444e+00,  7.5358e+00,  8.2118e+00,  7.3556e+00,
#           7.2021e+00,  8.3449e+00,  6.8042e+00,  8.3942e+00,  6.3627e+00,
#           7.7395e+00,  7.3103e+00,  5.4149e+00,  6.9211e+00,  7.5676e+00,
#           8.0616e+00,  7.3416e+00],
#         [ 3.2941e+00,  3.5773e+00,  3.3619e+00,  4.6832e+00,  4.8582e+00,
#           4.9631e+00,  4.2882e+00,  3.9852e+00,  4.1469e+00,  3.3927e+00,
#           3.1162e+00,  3.7959e+00,  4.3198e+00,  2.6789e+00,  2.3293e+00,
#           4.1583e+00,  3.7948e+00,  5.0970e+00,  3.9509e+00,  3.6691e+00,
#           4.3196e+00,  4.9685e+00,  5.7614e+00,  5.8441e+00,  3.9597e+00,
#           5.7724e+00,  2.6780e+00,  3.3114e+00,  3.5544e+00,  2.2871e+00,
#           3.3534e+00,  4.7381e+00],
#         [ 9.3018e+00,  6.6571e+00,  8.0484e+00,  7.8331e+00,  6.5024e+00,
#           6.8570e+00,  8.3792e+00,  9.4402e+00,  9.1481e+00,  8.8194e+00,
#           8.9075e+00,  8.0218e+00,  9.3118e+00,  7.3988e+00,  7.1450e+00,
#           8.6869e+00,  7.5152e+00,  8.3800e+00,  9.4465e+00,  8.7864e+00,
#           6.6192e+00,  8.1760e+00,  7.0731e+00,  7.9565e+00,  7.0018e+00,
#           7.8826e+00,  7.9066e+00,  6.1065e+00,  7.6182e+00,  9.0784e+00,
#           6.3820e+00,  8.2602e+00],
#         [-1.1687e+00, -3.5318e-01, -1.9066e+00, -3.3837e-02,  2.1576e+00,
#           5.6160e-01,  1.2496e+00, -3.1914e+00, -1.0080e+00,  3.5614e-01,
#          -1.8753e+00, -6.0190e-02, -2.1891e-01,  5.6370e-01,  1.6574e+00,
#          -7.9169e-01, -1.9504e-01, -4.2601e-01,  5.9452e-01, -5.2166e-01,
#           7.6944e-01,  3.2685e-01,  9.0304e-02,  9.8800e-01,  1.9025e-01,
#           9.8521e-01,  1.6765e+00,  9.0638e-01, -3.9567e-01, -4.2150e-01,
#          -3.1533e-01,  8.8739e-01],
#         [ 6.7549e+00,  7.1386e+00,  7.4400e+00,  6.2180e+00,  8.1137e+00,
#           7.8783e+00,  6.8722e+00,  5.3733e+00,  6.0474e+00,  6.9689e+00,
#           8.9186e+00,  5.7883e+00,  7.2209e+00,  6.0148e+00,  8.3184e+00,
#           6.8608e+00,  5.9135e+00,  6.3168e+00,  6.2431e+00,  6.7610e+00,
#           6.1219e+00,  6.9623e+00,  7.0686e+00,  6.8186e+00,  7.1673e+00,
#           7.2633e+00,  5.2348e+00,  7.3099e+00,  7.1388e+00,  6.0323e+00,
#           8.8953e+00,  8.0651e+00],
#         [ 2.6014e+00,  1.3847e+00,  2.4943e+00,  2.3977e+00,  2.6478e+00,
#           4.6353e+00,  4.6308e+00, -1.0632e+00,  3.0471e+00,  3.6453e+00,
#           4.0007e+00,  2.7566e+00,  5.4528e+00,  8.4473e-01,  3.1665e+00,
#           2.6032e+00,  2.5429e+00,  3.3953e+00,  2.8129e+00,  3.7627e+00,
#           1.4869e+00,  4.3776e+00,  4.3339e+00,  8.5732e-01,  3.2353e+00,
#           2.6638e+00,  3.0670e+00,  4.1390e+00,  3.0270e+00,  2.0974e+00,
#           2.4769e+00,  9.1761e-01],
#         [-2.0769e-01, -6.4261e-01,  2.0164e-01, -2.1195e-02, -1.7468e+00,
#          -1.7510e-01, -3.0128e-01, -1.0089e-01,  5.0110e-02,  4.7462e-01,
#           2.1984e-01, -1.9733e+00, -9.0133e-01, -1.4935e+00,  8.2389e-01,
#           6.8341e-03, -2.2790e-01, -5.5404e-01, -5.4600e-01, -2.0622e+00,
#          -3.5698e-02, -9.0693e-01, -6.7537e-01,  2.9980e-01,  6.8257e-01,
#           1.2038e+00, -1.5436e+00,  1.7787e-03, -1.3860e+00, -3.9505e-01,
#          -9.3280e-01,  7.5317e-01]])
# dataset.edge_index = tensor([[ 0,  0,  0,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  3,  3,  3,
#           3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,
#           5,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  8,  8,  8,  8,  8,  8,  9,
#           9,  9,  9, 10, 10, 10, 10, 10, 10, 10],
#         [ 3,  6, 10,  2,  4,  5,  6,  8, 10,  1,  3,  4,  5,  8,  9,  0,  2,  4,
#           5,  6,  8, 10,  1,  2,  3,  5,  6,  7,  9,  1,  2,  3,  4,  6,  7,  8,
#          10,  0,  1,  3,  4,  5,  8, 10,  4,  5,  9,  1,  2,  3,  5,  6, 10,  2,
#           4,  7, 10,  0,  1,  3,  5,  6,  8,  9]])
# dataset.edge_attr =  None


In [7]:

%load_ext autoreload
%autoreload 2

ample = Ample()
ample.sim = True

# model = GraphLam_Model(<parameters>)
model.to_device('ample',data=inputs) #Change 

# out = model(*inputs)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Moving model to Ample
Compiling model
model name ToyModel
saving ToyModel(
  (layers): ModuleList(
    (0-1): 2 x MLP_Model(
      (layers): ModuleList(
        (0): Linear(in_features=32, out_features=32, bias=False)
      )
    )
  )
  (linear1): MLP_Model(
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=False)
    )
  )
  (linear2): MLP_Model(
    (layers): ModuleList(
      (0): Linear(in_features=32, out_features=32, bias=False)
    )
  )
)
tensor([[ 9.6883, 11.1025,  9.8591,  8.0255,  9.6129,  9.1705,  8.4635,  8.1544,
          7.1679,  9.9791,  9.5058,  9.4450,  7.7037,  9.6371,  8.6707,  8.8444,
          9.9860,  7.3060,  9.7810,  8.6714,  8.9157,  8.2921,  8.6613,  9.8056,
         10.2106,  9.4653,  9.7499,  6.8711,  8.6038, 11.3398,  7.6130,  8.7171],
        [ 1.1612,  1.9904,  0.9164,  1.6336,  1.7422,  0.0230,  1.6457,  1.3090,
          0.3629,  3.8

10it [00:00, 46655.22it/s]
10it [00:00, 52038.51it/s]

Updated configuration successfully written back to file.
Overloading


In [8]:
out = model(*inputs)

Executing on AMPLE
cd /home/aw1223/ip/agile/hw/sim
==== Running command: cd /home/aw1223/ip/agile/hw/sim; make run_sim
make -f Makefile.modelsim run_sim
make[1]: Entering directory '/home/aw1223/ip/agile/hw/sim'
set -o pipefail; LIBPYTHON_LOC=/home/aw1223/anaconda3/envs/ample/lib/libpython3.11.so.1.0 MODULE=runner TESTCASE= TOPLEVEL="work.top_wrapper" \
GPI_EXTRA= TOPLEVEL_LANG=verilog \
 /mnt/applications/mentor/modelsim-2019.2/modelsim/modeltech/linux_x86_64/vsim -c -64 -l sim_build/modelsim.log -onfinish exit -do sim_build/../simulate_no_gui.do -quiet -suppress vsim-3015  -pli /home/aw1223/anaconda3/envs/ample/lib/python3.11/site-packages/cocotb/libs/libcocotbvpi_modelsim.so \
 \
work.work_opt \
 2>&1 | tee sim_build/simulate.log
/mnt/applications/mentor/modelsim-2019.2/modelsim/modeltech/linux_x86_64/vish: /mnt/applications/Xilinx/19.2/Vitis/2019.2/lib/lnx64.o/libxml2.so.2: no version information available (required by /lib64/libfontconfig.so.1)
/mnt/applications/mentor/modelsim-20